In [1]:
import os
import numpy as np
import pandas as pd
import xarray
import matplotlib.pyplot as plt
import random
plt.rcParams.update({'font.size': 20})

In [2]:
def extract_channel(dataframe, channel):
    return dataframe.loc[dataframe['vertco_reference_1@body'] == channel]
    

In [3]:
def get_allsky(df):
    return df["obsvalue@body"].values - df["fg_depar@body"].values - df["biascorr_fg@body"].values

In [4]:
def get_clearsky(df):
    return df["tbclear@radiance_body"].values

In [5]:
DF_06 = pd.read_table('~/Dendrite/Projects/AWS-325GHz/MWHS/output_06_2020.txt', delim_whitespace=True)
#print (DF.shape, type(DF), list(DF))
DF_05 = pd.read_table('~/Dendrite/Projects/AWS-325GHz/MWHS/output_05_2020.txt', delim_whitespace=True)
#print (DF.shape, type(DF), list(DF))

DF = DF_06.append(DF_05, ignore_index = True)

In [6]:
DF

,lat@hdr,lon@hdr,zenith@sat,lsm@modsurf,vertco_reference_1@body,obsvalue@body,biascorr_fg@body,tbclear@radiance_body,fg_depar@body
0,67.059898,161.912094,45.340000,1.0,1.0,275.440002,-1.685521,275.743744,3.173276
1,67.059898,161.912094,45.340000,1.0,2.0,232.270004,-3.342128,233.095230,2.471030
2,67.059898,161.912094,45.340000,1.0,3.0,227.139999,-2.087710,228.170441,1.026415
3,67.059898,161.912094,45.340000,1.0,4.0,226.479996,-1.606335,227.285767,0.769991
4,67.059898,161.912094,45.340000,1.0,5.0,228.429993,-2.137400,230.016235,0.664680
...,...,...,...,...,...,...,...,...,...
104913014,-77.566803,-149.066193,62.259998,1.0,5.0,206.110001,-2.982516,208.914932,0.176298
104913015,-77.566803,-149.066193,62.259998,1.0,6.0,212.020004,-2.947539,214.218109,0.748032
104913016,-77.566803,-149.066193,62.259998,1.0,7.0,228.350006,-4.098697,232.321625,0.125994
104913017,-77.566803,-149.066193,62.259998,1.0,8.0,231.520004,-3.495422,234.369308,0.644969


In [12]:
df = DF.loc[ (DF['zenith@sat'] <= 7.5)\
            & (DF['lat@hdr']<= 60.0) & (DF['lat@hdr']>= -60.0) ]

In [13]:
channel = 1
df.loc[df['vertco_reference_1@body'] == channel]

,lat@hdr,lon@hdr,zenith@sat,lsm@modsurf,vertco_reference_1@body,obsvalue@body,biascorr_fg@body,tbclear@radiance_body,fg_depar@body
645,-43.174500,-127.795097,4.59,0.000000,1.0,213.520004,-2.445658,203.861618,3.870162
1125,-42.384701,-126.848000,3.03,0.000000,1.0,209.679993,-2.305825,201.587814,-8.133781
1140,-42.529099,-127.814697,3.36,0.000000,1.0,210.259995,-2.349738,202.167099,-9.319522
2550,-28.411800,-132.054504,3.34,0.000000,1.0,232.630005,-1.654477,229.841995,4.014254
3000,-27.645100,-131.252197,4.25,0.000000,1.0,227.729996,-1.435227,226.367142,-0.845205
...,...,...,...,...,...,...,...,...,...
104909800,12.205700,-23.146299,5.74,0.000000,1.0,223.889999,-1.215122,222.137619,2.345602
104911270,12.268100,0.739700,6.75,1.000000,1.0,288.950012,-0.767535,290.279938,0.367377
104911615,11.562400,2.074400,5.74,1.000000,1.0,288.529999,-0.676050,290.944458,-1.414751
104911690,11.616600,0.741300,5.52,1.000000,1.0,288.190002,-0.699365,291.981537,-2.806565


In [27]:
TB_as = []
TB_cs = []

channels = np.arange(1, 16, 1)
for channel in channels:
    print (channel)
    TB = extract_channel(df, float(channel))
    TB_as.append(get_allsky(TB))
    TB_cs.append(get_clearsky(TB))

TB_as = np.stack(TB_as)
TB_cs = np.stack(TB_cs)
lsm = TB["lsm@modsurf"].values
lon = TB["lon@hdr"].values
lat = TB["lat@hdr"].values
zen = TB["zenith@sat"].values
TB = np.stack([TB_cs, TB_as])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [28]:
TB.shape

(2, 15, 341832)

In [29]:
cases = np.arange(0, TB.shape[2], 1)
sky = ['clearsky', 'allsky']
channel_id = np.arange(1, 16, 1)
TB_MWHS = xarray.DataArray(TB, coords = [sky, channel_id, cases], dims = ['sky','channel', 'cases'], name = 'TB')
TB_MWHS.attrs['LSM'] = lsm
TB_MWHS.attrs['lon'] = lon
TB_MWHS.attrs['lat'] = lat
TB_MWHS.to_netcdf('TB_MWHS.nc')

In [30]:
def TB_test_train(TB, randomList):
    TB_test = TB[:, :, randomList]
    TB_test.attrs['LSM'] = lsm[randomList]
    TB_test.attrs['lon'] = lon[randomList]
    TB_test.attrs['lat'] = lat[randomList]
    return TB_test

In [38]:
randomList = random.sample(range(0, len(cases)), len(cases))
lim = int(len(cases) * 0.32)

TB_MWHS_test = TB_test_train(TB_MWHS, randomList[:lim])
TB_MWHS_test.to_netcdf('TB_MWHS_test.nc', 'w')

TB_MWHS_train = TB_test_train(TB_MWHS, randomList[lim:])
TB_MWHS_train.to_netcdf('TB_MWHS_train.nc', 'w')
#TB_MWHS[:, :, randomList[lim:]].to_netcdf('TB_MWHS_train.nc', 'w')
#TB_MWHS[:, :, randomList[:lim]].to_netcdf('TB_MWHS_test.nc', 'w')

In [39]:
len(cases)-lim

232446

In [ ]:

bins1 = np.arange(200, 300, 1)
fig, ax = plt.subplots(1, 1)
hist = np.histogram(TB[0, 10, :],bins1 )
ax.plot(bins1[:-1], hist[0])
ax.set_yscale('log')

hist = np.histogram(TB[1, 10, :],bins1 )
ax.plot(bins1[:-1], hist[0])

## testing data

In [ ]:
TB_183 = extract_channel(DF, channel )
TB_as = get_allsky(TB_183)
TB_cs = get_clearsky(TB_183)

TB_183

In [ ]:
channel = 11
TB_183 = extract_channel(df, channel )
TB_as = get_allsky(TB_183)
TB_cs = get_clearsky(TB_183)



In [ ]:
bins = np.arange(200, 300, 0.5)
hist = np.histogram(TB_as,bins, density = True)
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
ax.plot(bins[:-1], hist[0])

hist = np.histogram(TB_cs, bins, density = True)
ax.plot(bins[:-1], hist[0])
ax.set_yscale('log')
ax.set_ylabel('freq')
ax.set_xlabel('TB[K]')
ax.set_title('channel-%s'%str(channel))
ax.legend(['allsky', 'clearsky'])
fig.savefig('TB.png', bbox_inches = 'tight')

In [ ]:
TB_MWHS_test.shape